# Scraping Amazon

In [42]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [43]:
!pip install webdriver-manager

Defaulting to user installation because normal site-packages is not writeable


In [44]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [45]:
!pip install amazoncaptcha

Defaulting to user installation because normal site-packages is not writeable


In [46]:
import pandas as pd 
import numpy as np 
import requests
import sys
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from amazoncaptcha import AmazonCaptcha

In [47]:
chrome_options = Options()
chrome_options.add_argument("--headless")  # Запуск в фоновом режиме (без открытия браузера)

In [48]:
driver = webdriver.Chrome() # options=chrome_options

url = 'https://www.amazon.com/'

driver.get(url)
time.sleep(3)
# time.sleep(10) # Т.к. надо вручную пройти капчу (ну либо деньги платить за сервисы для ее автономного прохождения)

In [49]:
def SolveCaptcha(driver):
    link = driver.find_element(By.XPATH, "//div[@class = 'a-row a-text-center']//img").get_attribute('src')

    captcha = AmazonCaptcha.fromlink(link)
    
    captcha_value = AmazonCaptcha.solve(captcha)
    
    clickable = driver.find_element(By.ID, 'captchacharacters')
    ActionChains(driver) \
        .click(clickable) \
        .perform()
    
    time.sleep(1)
    
    ActionChains(driver)\
            .send_keys(captcha_value)\
            .perform()
    
    clickable = driver.find_element(By.XPATH, "//button[@class = 'a-button-text']")
    ActionChains(driver) \
        .click(clickable) \
        .perform()
    return driver

In [50]:
driver = SolveCaptcha(driver)

In [51]:
def SetCountry(driver):
    clickable = driver.find_element(By.ID, 'nav-global-location-popover-link')
    ActionChains(driver) \
        .click(clickable) \
        .perform()
    
    time.sleep(1)
    
    clickable = driver.find_element(By.ID, 'GLUXZipUpdateInput')
    ActionChains(driver) \
        .click(clickable) \
        .perform()
    
    time.sleep(1)
    
    ActionChains(driver)\
            .send_keys("10001")\
            .perform()
    
    time.sleep(1)
    
    clickable = driver.find_element(By.ID, 'GLUXZipUpdate')
    ActionChains(driver) \
        .click(clickable) \
        .perform()
    
    time.sleep(1)
    
    ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    
    time.sleep(2)

    return driver

In [52]:
driver = SetCountry(driver)

In [53]:
clickable = driver.find_element(By.ID, 'nav-hamburger-menu')
ActionChains(driver) \
    .click(clickable) \
    .perform()

time.sleep(2)

In [54]:
html = driver.page_source

soup = BeautifulSoup(html, 'html.parser')

df = pd.DataFrame({'Category': [], 'Subcategory': [], 'Link': []})

for i in soup.find_all("ul", {'class': "hmenu hmenu-translateX-right"}):
    if int(i['data-menu-id']) > 11 and int(i['data-menu-id']) < 29:
        categ = i.find_all('div')[1].text
        if 'Whole Foods' in categ or 'Amazon' in categ or 'More to Explore' in categ:
            continue
        flag = False
        for j in i.find_all('li'):
            try:
                if int(j.find('div')['aria-level']) == 6:
                    if flag:
                        break
                    flag = True
            except:
                pass
            try:
                k = j.find('a')
                if k.text != 'main menu' and 'Amazon' not in k.text:
                    df.loc[len(df)] = [categ, k.text, url[0:-1] + k['href']]
            except:
                pass

In [55]:
df.head(20)

,Category,Subcategory,Link
0,"Clothing, Shoes, Jewelry & Watches",Women,https://www.amazon.com/gp/browse.html?node=714...
1,"Clothing, Shoes, Jewelry & Watches",Men,https://www.amazon.com/gp/browse.html?node=714...
2,"Clothing, Shoes, Jewelry & Watches",Girls,https://www.amazon.com/gp/browse.html?node=714...
3,"Clothing, Shoes, Jewelry & Watches",Boys,https://www.amazon.com/gp/browse.html?node=714...
4,"Clothing, Shoes, Jewelry & Watches",Baby,https://www.amazon.com/gp/browse.html?node=714...
5,"Clothing, Shoes, Jewelry & Watches",Luggage,https://www.amazon.com/gp/browse.html?node=947...
6,"Clothing, Shoes, Jewelry & Watches",Luxury Stores,https://www.amazon.com/luxurystores?ref_=nav_e...
7,"Clothing, Shoes, Jewelry & Watches",The Drop,https://www.amazon.comhttps://www.amazon.com/t...
8,"Clothing, Shoes, Jewelry & Watches",Sales & Deals,https://www.amazon.com/gp/browse.html?node=953...
9,Books,Books,https://www.amazon.com/gp/browse.html?node=283...


In [56]:
df.shape

(138, 3)

In [57]:
driver.quit()

In [58]:
'''
df.to_csv('Categories_&_links.csv')
'''

"\ndf.to_csv('Categories_&_links.csv')\n"

# Выбрали книги

In [60]:
df[(df['Subcategory'] == 'Books')]

,Category,Subcategory,Link
9,Books,Books,https://www.amazon.com/gp/browse.html?node=283...


In [61]:
df[(df['Subcategory'] == "Children's Books")]

,Category,Subcategory,Link
10,Books,Children's Books,https://www.amazon.com/gp/browse.html?node=4&r...


In [62]:
data = pd.DataFrame({'Title': [], 'Price': [], 'Currency':[], 'Rating': [], 'Number of reviews': [], 'Original Price': [], 'Main Genre': []})

In [63]:
def Start(c, category_name):
    driver = webdriver.Chrome() # options=chrome_options
    flag = True
    while flag:
        try:
            url = df[(df['Subcategory'] == c)]['Link'].iloc[0]
            
            driver.get(url)
            
            time.sleep(2)
            
            SetCountry(driver)
            driver.refresh()
            time.sleep(5)
            
            clickable = driver.find_element(By.ID, 'twotabsearchtextbox')
            
            time.sleep(1)
            
            ActionChains(driver) \
                .click(clickable) \
                .perform()
            
            time.sleep(1) 
            
            ActionChains(driver)\
                        .send_keys(category_name)\
                        .perform()
                
            time.sleep(1)
            
            clickable = driver.find_element(By.ID, 'nav-search-submit-button')
            ActionChains(driver) \
                .click(clickable) \
                .perform()
            flag = False
        except:
            driver = SolveCaptcha(driver)

    return driver

## Children's books

In [65]:
driver = Start("Children's Books", "Children's book")

In [66]:
for j in range(60):
    a = True
    flag = True
    while flag:
        try:
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            for i in soup.find_all('div', {'class': 'a-section a-spacing-small puis-padding-left-small puis-padding-right-small'}):
                title = i.find('h2', {'class': 'a-size-base-plus a-spacing-none a-color-base a-text-normal'}).find('span').text
                try:
                    price = i.find('span', {'class': 'a-price'}).find('span', {'class': 'a-offscreen'}).text
                    currency = price[0]
                    price = float(price[1::])
                except:
                    price = np.nan
                    currency = np.nan
                    price = np.nan
                    
                try:
                    rating = float(i.find('a', {'class': 'a-popover-trigger a-declarative'}).get('aria-label').split()[0])
                    num_of_rev = int(i.find('div', {'class': 's-csa-instrumentation-wrapper alf-search-csa-instrumentation-wrapper'}).find('a', {'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style'}).get('aria-label').split()[0].replace(',', ''))
                except:
                    rating = np.nan
                    num_of_rev = np.nan
                if price == 0:
                    try:
                        price = i.find('div', {'class': 'a-section a-spacing-none a-spacing-top-mini'}).find('div', {'class': 'a-row a-size-base a-color-secondary'}).find('span').split()[1]
                        currency = price[0]
                        price = float(price[1::])
                    except:
                        price = 0     
                
                try:
                    orig_price = float(i.find('span', {'class': 'a-price a-text-price'}).find('span', {'class': 'a-offscreen'}).text[1::])
                except:
                    orig_price = price
                data.loc[len(data)] = [title, price, currency, rating, num_of_rev, orig_price, "Children's Books"]
        
            clickable = driver.find_elements(By.CLASS_NAME, 's-list-item-margin-right-adjustment')[-1]
            ActionChains(driver) \
                .click(clickable) \
                .perform()
            
            time.sleep(3)
            flag = True
        except:
            print(f'error at: {j}')
            try:
                driver = SolveCaptcha(driver)
            except:
                a = False
                break
    if not a:
        break
            

error at: 0


In [67]:
data

,Title,Price,Currency,Rating,Number of reviews,Original Price,Main Genre
0,Otto's Tales: Let's Visit the Liberty Bell,13.00,$,4.7,17.0,13.00,Children's Books
1,Meeting Our Outer Space Neighbors: The Origin ...,7.99,$,NaN,NaN,7.99,Children's Books
2,Puddles Gets a New Home,0.00,$,5.0,24.0,0.00,Children's Books
3,The Rainbow Fish,5.04,$,4.8,6801.0,9.95,Children's Books
4,I Love You to the Moon and Back,4.92,$,4.9,77328.0,7.99,Children's Books
...,...,...,...,...,...,...,...
3302,I Love You When You're Angry Coloring Book (Wi...,12.99,$,NaN,NaN,12.99,Children's Books
3303,"Bluey Tattle Tales - Activity Books for kids, ...",9.29,$,4.6,19.0,9.99,Children's Books
3304,Sunny the Bunny: A Mission For a Cleaner Planet,14.95,$,4.8,5.0,14.95,Children's Books
3305,"Unicorns on Vacation: A story among dreams,fri...",13.99,$,NaN,NaN,13.99,Children's Books


In [68]:
driver.quit()

In [69]:
def Scrape(n, driver, categ):
    for j in range(n):
        a = True
        flag = True
        while flag:
            try:
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                for i in soup.find_all('div', {'class': 'a-section a-spacing-small a-spacing-top-small'}):
                    title = i.find('h2', {'class': 'a-size-medium a-spacing-none a-color-base a-text-normal'}).find('span').text
                    try:
                        price = i.find('span', {'class': 'a-price'}).find('span', {'class': 'a-offscreen'}).text
                        currency = price[0]
                        price = float(price[1::])
                    except:
                        price = np.nan
                        currency = np.nan
                        price = np.nan
                        
                    try:
                        rating = float(i.find('a', {'class': 'a-popover-trigger a-declarative'}).get('aria-label').split()[0])
                        num_of_rev = int(i.find('div', {'class': 's-csa-instrumentation-wrapper alf-search-csa-instrumentation-wrapper'}).find('a', {'class': 'a-link-normal s-underline-text s-underline-link-text s-link-style'}).get('aria-label').split()[0].replace(',', ''))
                    except:
                        rating = np.nan
                        num_of_rev = np.nan
                    if price == 0:
                        try:
                            price = i.find('div', {'class': 'a-section a-spacing-none a-spacing-top-mini'}).find('div', {'class': 'a-row a-size-base a-color-secondary'}).find('span').split()[1]
                            currency = price[0]
                            price = float(price[1::])
                        except:
                            price = 0     
                    
                    try:
                        orig_price = float(i.find('span', {'class': 'a-price a-text-price'}).find('span', {'class': 'a-offscreen'}).text[1::])
                    except:
                        orig_price = price
                    data.loc[len(data)] = [title, price, currency, rating, num_of_rev, orig_price, categ]
            
                clickable = driver.find_elements(By.CLASS_NAME, 's-list-item-margin-right-adjustment')[-1]
                ActionChains(driver) \
                    .click(clickable) \
                    .perform()
                
                time.sleep(3)
                flag = False
            except:
                try:
                    driver = SolveCaptcha(driver)
                except:
                    a = False
                    break
        if not a:
            break
    return driver

In [70]:
fiction_categories = [
    "Comics & Manga",
    "Literature & Fiction",
    "Mystery, Thriller & Suspense",
    "Romance",
    "Science Fiction & Fantasy",
    "Teen & Young Adult"
]

nonfiction_categories = [
    "Arts & Photography",
    "Biographies & Memoirs",
    "Business & Money",
    "Calendars",
    "Computers & Technology",
    "Cookbooks, Food & Wine",
    "Crafts, Hobbies & Home",
    "Education & Teaching",
    "Engineering & Transportation",
    "Health, Fitness & Dieting",
    "History",
    "Humor & Entertainment",
    "Law",
    "Medical Books",
    "Parenting & Relationships",
    "Politics & Social Sciences",
    "Reference",
    "Religion & Spirituality",
    "Science & Math",
    "Self-help",
    "Sports & Outdoors",
    "Travel"
]

In [71]:
for categ in (fiction_categories + nonfiction_categories):
    print(categ)
    driver = Start("Books", categ)
    driver = Scrape(50, driver, categ)
    driver.quit()
    data.to_csv('All_added_books_data_temp.csv')

Comics & Manga
Literature & Fiction
Mystery, Thriller & Suspense
Romance
Science Fiction & Fantasy
Teen & Young Adult
Arts & Photography
Biographies & Memoirs
Business & Money
Calendars
Computers & Technology
Cookbooks, Food & Wine
Crafts, Hobbies & Home
Education & Teaching
Engineering & Transportation
Health, Fitness & Dieting
History
Humor & Entertainment
Law
Medical Books
Parenting & Relationships
Politics & Social Sciences
Reference
Religion & Spirituality
Science & Math
Self-help
Sports & Outdoors
Travel


In [72]:
data

,Title,Price,Currency,Rating,Number of reviews,Original Price,Main Genre
0,Otto's Tales: Let's Visit the Liberty Bell,13.00,$,4.7,17.0,13.00,Children's Books
1,Meeting Our Outer Space Neighbors: The Origin ...,7.99,$,NaN,NaN,7.99,Children's Books
2,Puddles Gets a New Home,0.00,$,5.0,24.0,0.00,Children's Books
3,The Rainbow Fish,5.04,$,4.8,6801.0,9.95,Children's Books
4,I Love You to the Moon and Back,4.92,$,4.9,77328.0,7.99,Children's Books
...,...,...,...,...,...,...,...
23763,Illinois State Park Passport Book: Explore 70 ...,12.99,$,3.5,8.0,12.99,Travel
23764,Top Secret: The Ultimate Invisible Ink Activit...,9.29,$,4.7,400.0,9.99,Travel
23765,Lonely Planet Sardinia (Travel Guide),17.34,$,4.4,152.0,21.99,Travel
23766,"Travel 3 Ways: $, $$, $$$$: Europe",2.99,$,4.8,20.0,2.99,Travel


In [109]:
data.to_csv('All_added_books_data.csv')

In [105]:
data.drop_duplicates()

,Title,Price,Currency,Rating,Number of reviews,Original Price,Main Genre
0,Otto's Tales: Let's Visit the Liberty Bell,13.00,$,4.7,17.0,13.00,Children's Books
1,Meeting Our Outer Space Neighbors: The Origin ...,7.99,$,NaN,NaN,7.99,Children's Books
2,Puddles Gets a New Home,0.00,$,5.0,24.0,0.00,Children's Books
3,The Rainbow Fish,5.04,$,4.8,6801.0,9.95,Children's Books
4,I Love You to the Moon and Back,4.92,$,4.9,77328.0,7.99,Children's Books
...,...,...,...,...,...,...,...
23741,Kids' Travel Guide - Spain: The fun way to dis...,11.95,$,4.4,93.0,11.95,Travel
23742,Travel Journal,NaN,NaN,4.4,55.0,NaN,Travel
23743,Illinois State Park Passport Book: Explore 70 ...,12.99,$,3.5,8.0,12.99,Travel
23744,Top Secret: The Ultimate Invisible Ink Activit...,9.29,$,4.7,400.0,9.99,Travel
